In [1]:
!cp -r ../pandata/uploads/twp-* assets/img


In [29]:
import sqlite3
conn = sqlite3.connect('../pandata/app.db')
c = conn.cursor()

c.execute("SELECT * FROM person_data")
entries = c.fetchall()



In [30]:
twerps = [_ for _ in entries if 'twp-' in _[0]]

In [33]:
import re
import os

TAGS = ['midwest', 'southwest', 'northeast', 'southeast', 'west']
def name(t, text):
    name =  t.split('twp-')[1]
    if any('list as:' in t for t in text):
        name = [_ for _ in text if 'list as:' in _][0].split(':')[1].strip()
    return name.replace('"', '')

def extract_tags(text):
    for t in TAGS:
        for line in text:
            if t in line:
                return t
    return "other"
    
def title(t, text):
    n = name(t, text)
    proper = re.sub(r"(\w)([A-Z])", r"\1 \2", n)
    if proper.split()[-1].lower() < 'g':
        alpha_tag = 'ag'
    elif proper.split()[-1].lower() < 'q':
        alpha_tag = 'hp'
    else:
        alpha_tag = 'qz'
    return """---
layout: default
title: "%s"
tags: [%s, %s]
---


## %s
""" % (proper, alpha_tag, extract_tags(text), proper)


def image_link(direc, fname):
    return """


![screenshot]({{ "/assets/img/%s/%s" | prepend:site.baseurl }})""" % (direc, fname)


def pdf_link(direc, fname):
    return """


[View pdf]({{ "/assets/img/%s/%s" | prepend:site.baseurl }})""" % (direc, fname)

    
def process(line, f):
    if "'dict'" in str(type(line)):
        line = dict(line)
        for k, v in line.items():
            if 'Gender' in k or 'Interested' in k or 'list as' in k:
                continue
            if type(v) == type([]):
                v = ' , '.join(v)
            f.write("%s: %s\n" % (k, v))
    elif 'tuple' in str(type(line)):
        for t in line:
            if type(t) == type('str'):
                f.write(t)
            else:
                t = [_ for _ in t if _]
                if len(t) < 2:
                    f.write(t[0])
                elif 'https' in t[1]:
                    f.write("[%s](%s) %s\n" % (t[0], t[1], ','.join(t[2:])))
                elif len(t) == 2:
                    f.write("%s: %s\n" % (t[1], t[0]))
                else:
                    f.write("%s: [%s](https://facebook.com/%s)\n" % (t[3], t[2], t[0]))
                
    else:
        print(type(line), item, '\n\n')
        
    
    
for t in twerps:
    if 'main' in t[0] or 'Tips' in t[0]: continue
    with open("_posts/2018-01-01-%s.md" % name(t[0], t[1]), 'w') as f:
        f.write(title(t[0], t[1].split('\n')))
        for line in t[1].split('\n'):
            if 'FB user ID' in line or "sad loser" in line:
                pass
            if any (t in line for t in TAGS) and len(line) < 20:
                pass
            else:
                f.write(line)
        
        if t[0] in os.listdir('assets/img'):
            files = os.listdir('assets/img/%s' % t[0])
            files.sort(key=lambda x: '.txt' not in x)
            for fname in files:
                r = fname.replace(" ", "").replace("(", "").replace(")", "")
                if( r != fname):
                    os.rename('assets/img/%s/%s' % (t[0], fname), 'assets/img/%s/%s' %  (t[0], r))
                    fname = r

                if '.txt' in fname:
                    
                    f2 = open('assets/img/%s/%s' % (t[0], fname), 'rb')
                    for i, line in enumerate(f2):
                        if i < 2:
                            continue
                    
                        line = line.decode('utf-8', 'ignore')
                        if 'utomatically' in line or 'manual research here' in line:
                            pass
                        elif 'ID number' in line or 'list as' in line:
                            pass
                        elif 'Facebook' in line:
                            split = max(line.find(':') + 1, 0)
                            line = line[split:]
                            if len(line.split()) == 1:
                                continue
                            try:
                                process(eval(line), f)
                            except SyntaxError:
                                f.write(line)
                        else:
                            f.write(line)
                        
                elif '.pdf' in fname:
                    f.write(pdf_link(t[0], fname))

                else:
                    f.write(image_link(t[0], fname))
        else:
            print(t[0])

            
!rm _posts/2018-01-01-SearchTips.md 
!rm _posts/2018-01-01-SönnyThomas.md 

twp-SönnyThomas
rm: _posts/2018-01-01-SearchTips.md: No such file or directory


In [9]:
from pdf2image import convert_from_path
import PIL
from PIL import Image
for t in twerps:
    if 'main' in t[0]:
        contine
        
    with open("_posts/2018-01-01-%s.md" % name(t[0]), 'w') as f:
        f.write(title(t[0], t[1]))
        f.write(t[1])
        
        if t[0] in os.listdir('assets/img'):
            for fname in os.listdir('assets/img/%s' % t[0]):
                r = fname.replace(" ", "").replace("(", "").replace(")", "")
                if(r != fname):
                    os.rename('assets/img/%s/%s' % (t[0], fname), 'assets/img/%s/%s' %  (t[0], r))
                    fname = r

                if '.pdf' in fname:
                    images = convert_from_path('assets/img/%s/%s' % (t[0], fname))
                    base = fname.split('.')[0]
                    for num, i in enumerate(images):
                        (height, width) = i.size
                        i = i.resize((int(height/2), int(width/2)), PIL.Image.ANTIALIAS)
                        i.save('assets/img/%s/%s%s.png' % (t[0], base, num))

                elif 'txt' not in fname:
                    image_file = open('assets/img/%s/%s' %  (t[0], fname), 'rb')    
                    image = Image.open(image_file)

                    # next 3 lines strip exif
                    data = list(image.getdata())
                    image_without_exif = Image.new(image.mode, image.size)
                    image_without_exif.putdata(data)

                    image_without_exif.save('assets/img/%s/%s' %  (t[0], fname))

<_io.BufferedReader name='assets/img/twp-JustinBurger/JustinBurgeriscontinuingonwithNationalistInitiative.jpg'>
<_io.BufferedReader name='assets/img/twp-JustinBurger/JustinBurger1.jpg'>
<_io.BufferedReader name='assets/img/twp-JustinBurger/JustinBurger3.jpg'>
<_io.BufferedReader name='assets/img/twp-JustinBurger/JustinBurger2.jpg'>
<_io.BufferedReader name='assets/img/twp-JustinBurger/JustinBurger4.jpg'>
<_io.BufferedReader name='assets/img/twp-JustinBurger/JustinBurgerisTWPOfficerofNorthGeorgia.jpg'>
<_io.BufferedReader name='assets/img/twp-JustinBurger/JustinBurgerwasinShelbyville.jpg'>
<_io.BufferedReader name='assets/img/twp-JustinBurger/JustinBurgerlivesinDouglasville.jpg'>
<_io.BufferedReader name='assets/img/twp-CoryPruchinsky/CoryPruchinsky.jpg'>
<_io.BufferedReader name='assets/img/twp-CoryPruchinsky/CoryPruchinskywantstojoinrebrandedgroup.jpg'>
<_io.BufferedReader name='assets/img/twp-NatsirtSsab/NatsirtSsabwithMatthewHeimbach.jpg'>
<_io.BufferedReader name='assets/img/twp-Na

<_io.BufferedReader name='assets/img/twp-JosephHutchins/Maigann_Hutchins2.jpg'>
<_io.BufferedReader name='assets/img/twp-JosephHutchins/Joseph_Hutchins_facebook_assault.jpg'>
<_io.BufferedReader name='assets/img/twp-JosephHutchins/Joseph_Hutchins_Nazi_riot.jpg'>
<_io.BufferedReader name='assets/img/twp-CesarAdolfo/ScreenShot2018-03-18at10.05.56PM.png'>
<_io.BufferedReader name='assets/img/twp-CesarAdolfo/CesarAdolfo.png'>
<_io.BufferedReader name='assets/img/twp-CesarAdolfo/22090179_103762703712555_9084110885014735429_n.jpg'>
<_io.BufferedReader name='assets/img/twp-CesarAdolfo/ScreenShot2018-03-18at10.05.44PM.png'>
<_io.BufferedReader name='assets/img/twp-CesarAdolfo/CesarAdolfowithMikeEnoch.jpg'>
<_io.BufferedReader name='assets/img/twp-CesarAdolfo/CesarAdolfomemberofTWP.jpg'>
<_io.BufferedReader name='assets/img/twp-BrookeWhaley/BrookeWhaleylikelymemberTWP.jpg'>
<_io.BufferedReader name='assets/img/twp-BrookeWhaley/BrookeWhaleypartofagroup1.jpg'>
<_io.BufferedReader name='assets/img

<_io.BufferedReader name='assets/img/twp-ChristopherCope/ChristopherCope6.jpg'>
<_io.BufferedReader name='assets/img/twp-ChristopherCope/ChristopherCope10.jpg'>
<_io.BufferedReader name='assets/img/twp-ChristopherCope/ChristopherCope11.jpg'>
<_io.BufferedReader name='assets/img/twp-ChristopherCope/ChristopherCopedeploymentinfo.jpg'>
<_io.BufferedReader name='assets/img/twp-ChristopherCope/ChristopherCopelivesnearnorthTX.jpg'>
<_io.BufferedReader name='assets/img/twp-ChristopherCope/ChristopherCopeismemberofTWP.jpg'>
<_io.BufferedReader name='assets/img/twp-ChristopherCope/ChristopherCopejoinedtheNavy.jpg'>
<_io.BufferedReader name='assets/img/twp-ChristopherCope/ChristopherCope8.jpg'>
<_io.BufferedReader name='assets/img/twp-ChristopherCope/ChristopherCope9.jpg'>
<_io.BufferedReader name='assets/img/twp-ChristopherCope/ChristopherCopenavygraduation.jpg'>
<_io.BufferedReader name='assets/img/twp-ChristopherCope/ChristopherCopepossibleTWP.jpg'>
<_io.BufferedReader name='assets/img/twp-Jo

<_io.BufferedReader name='assets/img/twp-AdamTuttle/AdamTuttleleft.jpg'>
<_io.BufferedReader name='assets/img/twp-AdamTuttle/AdamTuttlemanagesullrmediapage2.jpg'>
<_io.BufferedReader name='assets/img/twp-SonnyThomas/SonnyThomasonright.jpg'>
<_io.BufferedReader name='assets/img/twp-SonnyThomas/SonnyThomas.jpg'>
<_io.BufferedReader name='assets/img/twp-SonnyThomas/untitled30.png'>
<_io.BufferedReader name='assets/img/twp-SonnyThomas/SonnyThomaspromotingTWP.jpg'>
<_io.BufferedReader name='assets/img/twp-SonnyThomas/SonnyThomas1.png'>
<_io.BufferedReader name='assets/img/twp-SonnyThomas/SonnyThomas0.png'>
<_io.BufferedReader name='assets/img/twp-SonnyThomas/SonnyThomas2.png'>
<_io.BufferedReader name='assets/img/twp-SonnyThomas/SonnyThomas2.jpg'>
<_io.BufferedReader name='assets/img/twp-SonnyThomas/SonnyThomas3.jpg'>
<_io.BufferedReader name='assets/img/twp-SonnyThomas/SonnyThomas3.png'>
<_io.BufferedReader name='assets/img/twp-SonnyThomas/SonnyThomas4.png'>
<_io.BufferedReader name='asset

NameError: name 'contine' is not defined

rm: _posts/2018-01-01-SearchTips.md: No such file or directory
